In [161]:
import re
import pandas as pd 
from collections import Counter


def load_data(file_path):
    """Cargar datos desde un archivo CSV."""
    data = pd.read_csv(file_path, header=None)
    column_names = ['Tipo', 'Subtipo', 'Descripcion', 'Columna4', 'Columna5', 'Columna6', 'Columna7', 'Columna8']
    data.columns = column_names
    # Eliminar las columnas 4, 5, 6 y 7
    data = data.drop(['Columna4', 'Columna5', 'Columna6', 'Columna7','Columna8'], axis=1)
    return data



In [162]:
def clean_description(text):
    """Limpiar el texto y devolver una lista de palabras."""
    if not isinstance(text, str):
        text = ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Eliminar caracteres especiales
    words = text.split()
    
    # Lista de palabras vacías (stopwords) que no queremos contar
    stoplist = set([
        'de', 'con', 'y', 'sin', 'que', 'o', 'a', 'la', 'al', 'un', 'una', 
        'los', 'las', 'por', 'en', 'no', 'n.d', 'para', 's/m', 's/n', 
        'el', 'del', 'x/la', 'a', 'es','x','gb'
    ])
    words = [word for word in words if word not in stoplist]
    return words

In [163]:
def get_top_words(data, n=10):
    """Obtener las 'n' palabras más comunes en las descripciones."""
    # Unir todas las listas de palabras de las descripciones
    all_words = [word for description in data['Descripcion_Limpia'] for word in description]
    
    # Contar la frecuencia de cada palabra
    word_counts = Counter(all_words)
    
    # Seleccionar las 'n' palabras más comunes
    top_words = [word for word, count in word_counts.most_common(n)]
    return top_words

In [164]:
def process_and_classify(data, n_top_words=10):
    """Añadir columnas para las 'n' palabras más comunes en la descripción."""
    # Limpiar la columna 'Descripcion' y crear una lista de palabras
    data['Descripcion_Limpia'] = data['Descripcion'].apply(clean_description)

    # Obtener las palabras más comunes
    top_words = get_top_words(data, n_top_words)
    
    # Crear una columna para cada una de las palabras más comunes
    for word in top_words:
        data[word] = data['Descripcion_Limpia'].apply(lambda desc: 1 if word in desc else 0)

    # Eliminar la columna auxiliar 'Descripcion_Limpia'
    data = data.drop(columns=['Descripcion_Limpia'])

    return data

In [165]:
def add_classification_columns(data):
    """Añadir columnas para las clasificaciones predefinidas en 'Tipo' y 'Subtipo'."""
    default_types = ['mobiliario', 'equipo', 'transporte', 'libro']
    
    # Crear una columna para cada tipo predefinido
    for tipo in default_types:
        data[f'{tipo}'] = data.apply(lambda row: 1 if isinstance(row['Tipo'], str) and tipo in row['Tipo'].lower() 
                                               or isinstance(row['Subtipo'], str) and tipo in row['Subtipo'].lower() 
                                               else 0, axis=1)


In [166]:
def process_and_classify(data, n_top_words=10):
    """Añadir columnas para las 'n' palabras más comunes en la descripción y las clasificaciones predefinidas."""
    # Limpiar la columna 'Descripcion' y crear una lista de palabras
    data['Descripcion_Limpia'] = data['Descripcion'].apply(clean_description)

    # Obtener las palabras más comunes
    top_words = get_top_words(data, n_top_words)
    
    # Crear una columna para cada una de las palabras más comunes
    for word in top_words:
        data[word] = data['Descripcion_Limpia'].apply(lambda desc: 1 if word in desc else 0)

    # Añadir las columnas de clasificación basadas en 'Tipo' y 'Subtipo'
    add_classification_columns(data)

    # Eliminar la columna auxiliar 'Descripcion_Limpia'
    data = data.drop(columns=['Descripcion_Limpia'])

    return data

In [167]:
def filter_items(data, search_term):
    """Filtrar artículos que contengan el término de búsqueda en la descripción."""
    search_term = search_term.lower()
    filtered_data = data[data['Descripcion'].apply(lambda x: search_term in x.lower())]
    return filtered_data

In [168]:
def save_results(data, output_file):
    """Guardar el DataFrame con las nuevas columnas en un archivo CSV."""
    data.to_csv(output_file, index=False)
    print(f"Resultados guardados en '{output_file}'")

In [169]:
if __name__ == "__main__":
    # Cargar los datos
    data = load_data('Actividad10000.csv')

    # Procesar y clasificar los datos tomando las 20 palabras más comunes
    data = process_and_classify(data, n_top_words=20)

    # Mostrar todas las columnas generadas
    print("Columnas generadas:")
    print(data.columns)

    # Mostrar el número total de columnas generadas
    print(f"Total de atributos generados: {len(data.columns)}")

    # Guardar resultados
    save_results(data, 'Clasificacion_Extendida.csv')

Columnas generadas:
Index(['Tipo', 'Subtipo', 'Descripcion', 'color', 'negro', 'libro',
       'computadora', 'madera', 'monitor', 'hp', 'mesa', 'factura', 'marca',
       'silla', 'modelo', 'blanco', 'escritorio', 'cpu', 'cm', 'archivero',
       'gris', 'disco', 'azul', 'mobiliario', 'equipo', 'transporte'],
      dtype='object')
Total de atributos generados: 26
Resultados guardados en 'Clasificacion_Extendida.csv'
